In [1]:
import os
from pathlib import Path
import shutil
from distutils.dir_util import copy_tree

import pandas as pd
import json
import time
import datetime
from PIL import Image

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import scipy.stats
import itertools

# https://github.com/qwirky-yuzu/COCO-to-YOLO/blob/main/coco_to_yolo.py
# https://github.com/pylabel-project/samples/blob/main/coco2yolov5.ipynb
try:
    import pylabel
except:
    !pip install pylabel
finally:
    from pylabel import importer

import matplotlib.patches as patches
import matplotlib
import matplotlib.colors as mcolors

plt.rcParams.update({'font.size': 22})
style_label = 'fivethirtyeight' #plt.style.available 

In [3]:
import labelutilits as utl

In [4]:
try:
    import dsatools 
except:
    !pip install dsatools
finally:
    import dsatools.utilits as ut

In [5]:
path = os.getcwd() 
print(utl.list_dirs(path))
anno_path = utl.get_anno_path(path,'train')
anno_path

['labelutilits', 'part1', 'part2', 'part3', 'part4', 'part5', 'part6', 'part7', 'part8', 'part9', 'train']


'D:\\ASBEST!\\ALLVEINS\\train\\annotation.json'

In [6]:
utl.anno_info(anno_path)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


{'name': 'train',
 'anno_path': 'D:\\ASBEST!\\ALLVEINS\\train\\annotation.json',
 'anno_fname': 'annotation.json',
 'image_dir_path': 'D:\\ASBEST!\\ALLVEINS\\train',
 'length': 249,
 'anno_number': 4237,
 'class_id': [1, 2],
 'class_names': ['asbest', 'stone'],
 'height': [2048, 3456],
 'width': [2592, 5184],
 'COCO_obj': <pycocotools.coco.COCO at 0x2351f843a58>,
 'image_fname_example': 'D:\\ASBEST!\\ALLVEINS\\train\\14-02-21_5_1_9_part1.bmp'}

In [10]:
anno = utl.Annotation(anno_path).\
        set_cat_names(new_names = ['stones', 'asbest']).\
        filter_cat(cat_ids=[2]).\
        replace_image_dir().rest_ids().\
        correct_images_size().save('veins_anno.json')
anno.info()

{'cat_ids': [1],
 'class_names': ['asbest'],
 'supercategory': [''],
 'width': [2592],
 'height': [2048],
 'length': 203,
 'anno_number': 3729,
 'fname_example': 'D:\\ASBEST!\\ALLVEINS\\train\\14-02-21_5_1_9_part1.bmp',
 'image_dir_path': 'D:\\ASBEST!\\ALLVEINS\\train',
 'dataset_name': 'ALLVEINS',
 'anno_path': 'D:\\ASBEST!\\ALLVEINS\\train\\annotation.json',
 'anno_fname': 'annotation.json'}

In [13]:
data = anno.data

In [11]:
anno = utl.Annotation(anno_path).\
        set_cat_names(new_names = ['stones', 'asbest']).\
        filter_cat(cat_ids=[1]).\
        replace_image_dir().\
        correct_images_size().rest_ids().save('stone_anno.json')
anno.info()

{'cat_ids': [1],
 'class_names': ['stones'],
 'supercategory': [''],
 'width': [2592],
 'height': [2048],
 'length': 102,
 'anno_number': 508,
 'fname_example': 'D:\\ASBEST!\\ALLVEINS\\train\\11-11-2021-12-25-32_1_1_63_detailed_part7.bmp',
 'image_dir_path': 'D:\\ASBEST!\\ALLVEINS\\train',
 'dataset_name': 'ALLVEINS',
 'anno_path': 'D:\\ASBEST!\\ALLVEINS\\train\\annotation.json',
 'anno_fname': 'annotation.json'}

In [24]:
with open(anno_path) as json_file:
    data = json.load(json_file)
    json_file.close()